In [1]:
import numpy as np
import torch
from transformers import *
from sklearn.mixture import GaussianMixture
import time
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

You are using torchaudio==0.9.0, but torchaudio>=0.10.0 is required to use MCTCTFeatureExtractor. This requires torch>=1.10.0. Please upgrade torch and torchaudio.
You are using torch==1.9.0+cu111, but torch>=1.10.0 is required to use ViltModel. Please upgrade torch.


cuda


In [2]:
train_embeddings = {}
test_embeddings = {}
sents = np.load('distilBERT/sentences.npy')
vectors = np.load('distilBERT/vectors.npy')
domain = np.load('distilBERT/domain.npy')
sentiment = np.load('distilBERT/sentiment.npy')

split = int(0.8 * len(sents))
train_embeddings['sents'] = sents[:split]
train_embeddings['vectors'] = vectors[:split]
train_embeddings['domain'] = domain[:split]
train_embeddings['sentiment'] = sentiment[:split]
test_embeddings['sents'] = sents[split:]
test_embeddings['vectors'] = vectors[split:]
test_embeddings['domain'] = domain[split:]
test_embeddings['sentiment'] = sentiment[split:]

In [3]:
DOMAINS = ['Automotive', 'Books', 'Music', 'Software', 'Baby']
N_CLUSTERS = 10

def GMM(data, num_clusters):
    model = GaussianMixture(
        n_components=num_clusters, 
        covariance_type='full', 
        max_iter=100, 
        random_state=0,
        verbose=1)

    model.fit(data)
    return model


def get_predictions(model, test_data):
    return model.predict(test_data)


def assign_labels_to_clusters(true_labels, predictions, num_clusters=8):
    cluster_assignment = {}
    true_count = 0
    total = 0
    for k in range(num_clusters):
        count = [0] * len(DOMAINS)
        for i, pred in enumerate(predictions):
            if pred == k:
                count[DOMAINS.index(true_labels[i])] += 1
        max_count = max(count)
        total_count = sum(count)
        cluster_assignment[k] = (DOMAINS[count.index(max_count)], max_count, total_count)
        true_count += max_count
        total += total_count
    acc = true_count / total
    return cluster_assignment, acc


def calc_test_accuracy(true_labels, predictions, cluster_assignment):
    total = 0
    true_count = 0
    for i, pred in enumerate(predictions):
        if cluster_assignment[pred][0] == true_labels[i]:
            true_count +=1
        total += 1
    return true_count / total

In [ ]:
start = time.time()
model = GMM(train_embeddings['vectors'], N_CLUSTERS)
end  = time.time()
print(f"Time taken to converge: {(end-start) / 3600} hrs.")

In [ ]:
train_pred = get_predictions(model, train_embeddings['vectors'])
cluster_assignment, accuracy = assign_labels_to_clusters(train_embeddings['domain'], train_pred, N_CLUSTERS)
print(f"Train accuracy is {accuracy:.4f}")

test_pred = get_predictions(model, test_embeddings['vectors'])
test_accuracy = calc_test_accuracy(test_embeddings['domain'], test_pred, cluster_assignment)
print(f"Test accuracy is {test_accuracy:.4f}")

np.save('test_pred_gmm_k5_distilBERT.npy', test_pred)

In [ ]:
for k in range(N_CLUSTERS):
    print(f'Cluster-{k+1} | label: {cluster_assignment[k][0]} | Purity: {cluster_assignment[k][1]/cluster_assignment[k][2]}')

In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd

tsne = TSNE(n_components=2, verbose=1, random_state=1)
z = tsne.fit_transform(test_embeddings['vectors'])

In [ ]:
df = pd.DataFrame()
df["labels"] = test_embeddings['domain']
df["clusters"] = [cluster_assignment[pred][0] for pred in test_pred]
# df["clusters"] = test_pred
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]
sns.scatterplot(data=df, x="comp-1", y="comp-2", hue="labels", palette=sns.color_palette("hls", 5))

In [ ]:
sns.scatterplot(data=df, x="comp-1", y="comp-2", hue="clusters", palette=sns.color_palette("hls", 5))